Following things are implemented in this notebook

1. Making use of auxiliary variables obtained from Hydrated tweets, like favorite_count, possibly__sensitive, retweet_count, user_verified etc.
2. Glove 27B Tweet Embeddings [50d,100d]
3. Different model tryouts of SVM, MNB, RandomForest, LGBM, XGBoost, BiLSTM, Fasttext with GloVe Twitter embeddings

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io
pd.set_option('display.max_colwidth', -1)
!pip install emoji
! pip install -q scikit-plot
import scikitplot as skplt

import emoji
import gensim
from gensim import corpora, models, similarities

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

import re
import os
import sys
import string
import time
import math
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

import unicodedata as uni
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D,GRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.optimizers import Adam, RMSprop, Adamax
from keras.layers import MaxPooling2D,Activation,Dropout,Flatten,Dense,BatchNormalization

print(os.listdir("../"))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['etc', 'media', 'bin', 'srv', 'sys', 'boot', 'home', 'lib', 'dev', 'opt', 'var', 'proc', 'usr', 'tmp', 'lib64', 'root', 'mnt', 'sbin', 'run', 'content', '.dockerenv', 'tools', 'datalab', 'swift', 'dlib-19.18.0-cp36-cp36m-linux_x86_64.whl', 'dlib-19.18.0-cp27-cp27mu-linux_x86_64.whl', 'tensorflow-1.15.2', 'lib32']


In [ ]:
ls

auxiliary_test.csv   merged_tweets.txt  sample_data/
auxiliary_train.csv  MeTooMMD_test.csv  test.txt


In [ ]:
df = pd.read_csv("merged_tweets.txt")
aux_train = pd.read_csv("/content/auxiliary_train.csv")

df = df.drop(columns = ['Unnamed: 0'],axis=1)

flag_cols = ['Text_Only_Informative', 'Image_Only_Informative', 'Directed_Hate', 'Generalized_Hate', 'Sarcasm', 'Allegation', 'Justification', 
             'Refutation', 'Support', 'Oppose']
for i in flag_cols:
  df[i] = df[i].map({1.0: 1, 0.0: 0})
  df[i] = df[i].astype('uint8')

train_df = pd.concat([df,aux_train],axis=1)

#Some tweet pre-proc
def normalise_text (text):
    text = text.str.lower() # lowercase
    text = text.str.replace(r"\\n"," ")
    text = text.str.replace(r"\\n\n"," ")
    text = text.str.replace(r"\n\n"," ")
    text = text.str.replace(r"\\\n\\n"," ")
    return text

train_df["Tweets"]=normalise_text(train_df["Tweets"])
train_df['possibly__senstive'] = train_df['possibly__senstive'].astype('uint8')
train_df['user_verified'] = train_df['user_verified'].astype('uint8') 

In [ ]:
test = pd.read_csv('test.txt')

for i in range(len(test.tweets)):
    test['tweets'].iloc[i] = test['tweets'].iloc[i].replace('"', '')

aux_test = pd.read_csv("/content/auxiliary_test.csv")
test_df = pd.concat([test,aux_test],axis=1)
test_df["tweets"]=normalise_text(test_df["tweets"])
test_df['possibly__senstive'] = test_df['possibly__senstive'].astype('uint8')
test_df['user_verified'] = test_df['user_verified'].astype('uint8') 

test_df.sample(2)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,tweet_id,tweets,favorite_count,possibly__senstive,retweet_count,user_verified
406,1051894764050808842,"after a deluge of #metoo allegations, indian men claim they are the ones under attack https://t.co/e2pc0gsvdo https://t.co/0wmou9fucw",0,0,0,0
1299,1058278364270739457,"@chtycommission has updated its #safeguarding advice. the free cef safety module has been updated to reflect this &amp; also assesses #whistleblowing, #harassment &amp; other safety issues, using 50+ checks. join us at https://t.co/rakurcheys https://t.co/p4vp76hzui #metoo https://t.co/3biqovuwr8",0,0,0,0


### Text Pre-prop
1. Extracting & Counting Emojis
2. Similar Pre-processing as earlier versions
3. No stemming, No removal of stopwords

In [ ]:
#train_df = train_df.drop(columns = ['TweetId'],axis=1)
#test_df = test_df.drop(columns = ['tweet_id'],axis=1)
train_df.sample(2)##

,Tweets,TweetId,Text_Only_Informative,Image_Only_Informative,Directed_Hate,Generalized_Hate,Sarcasm,Allegation,Justification,Refutation,Support,Oppose,favorite_count,possibly__senstive,retweet_count,user_verified
6629,"ready to fight, says @priyaramani after #mjakbar sues her over #metoo. read here: https://t.co/iyfxypnnj2 #priyaramani #metooindia #meetooindia #metoocontroversy https://t.co/4vncjteqs6",1052166509793955840,1,1,0,0,0,0,0,1,0,1,0,0,0,0
1537,yesterday our howard university mentors and howard middle school mentees discussed the history of the #metoo moment and signed the girls inc. #girlstoo pledge! respect starts young! have you signed the pledge yet? visit https://t.co/prrznwz7ol to sign on! #girlsinc #girlstoo https://t.co/3dgellxxx1,1052227813523238917,1,1,0,0,0,0,0,0,0,0,13,0,6,0


In [ ]:
def extract_emojis(s):
  return ''.join(c for c in s if c in emoji.UNICODE_EMOJI)

D = []
L = list(train_df['Tweets'])

for i in range(len(L)):
  D.append(extract_emojis(L[i]))

train_df['emojis'] = pd.DataFrame(D)

In [ ]:
def extract_emojis(s):
  return ''.join(c for c in s if c in emoji.UNICODE_EMOJI)

D = []
L = list(test_df['tweets'])

for i in range(len(L)):
  D.append(extract_emojis(L[i]))

test_df['emojis'] = pd.DataFrame(D)

In [ ]:
# train_df = train_df.drop(['demojized_tweets'],axis=1)
train_df.iloc[6]

Tweets                    if labor win government what a #quinella 🏇🏼🏇🏼  prime minister - accused of historic rape allegation  deputy pm - married to convicted drug trafficker  #auspol #metoo #everest #kavanaugh @anncoulter @theboltreport @j_c_campbell @tomwconnell @sunriseon7 @annelisenews @couriermail https://t.co/h8jnmaz1zc
TweetId                   1049804220696748033                                                                                                                                                                                                                                                                                           
Text_Only_Informative     1                                                                                                                                                                                                                                                                                                             
Image_Only_In

In [ ]:
def analyze_tweet(tweet):
    result = {}
    result['MENTIONS'] = tweet.count('USER_MENTION')
    result['URLS'] = tweet.count('URL')
    #result['POS_EMOS'] = tweet.count('EMO_POS')
    #result['NEG_EMOS'] = tweet.count('EMO_NEG')
    tweet = tweet.replace('USER_MENTION', '').replace('URL', '')
    words = tweet.split()
    result['WORDS'] = len(words)
    bigrams = get_bigrams(words)
    result['BIGRAMS'] = len(bigrams)
    return result, words, bigrams

def get_bigrams(tweet_words):
    bigrams = []
    num_words = len(tweet_words)
    for i in range(num_words - 1):
        bigrams.append((tweet_words[i], tweet_words[i + 1]))
    return bigrams

def get_bigram_freqdist(bigrams):
    freq_dict = {}
    for bigram in bigrams:
        if freq_dict.get(bigram):
            freq_dict[bigram] += 1
        else:
            freq_dict[bigram] = 1
    counter = Counter(freq_dict)
    return counter

In [ ]:
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


def is_valid_word(word):
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', str(tweet))
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', str(tweet))
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', str(tweet))
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', str(tweet))
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', str(tweet))
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', str(tweet))
    return tweet

use_stemmer = False

def preprocess_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r"\\n","", str(tweet))
    tweet = re.sub(r"\n\n"," ", str(tweet))
    tweet = re.sub(r"\n","", str(tweet))
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', str(tweet))
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', str(tweet))
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', str(tweet))
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', str(tweet))
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', str(tweet))
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    #tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', str(tweet))
    words = tweet.split()

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            if use_stemmer:
                word = str(porter_stemmer.stem(word))
            processed_tweet.append(word)

    return ' '.join(processed_tweet)

In [ ]:
K = []
L = list(train_df['Tweets'])
for i in range(len(L)):
  K.append(preprocess_tweet(L[i]))

train_df['Processed_tweets'] = pd.DataFrame(K)
train_df.sample(2)

,Tweets,TweetId,Text_Only_Informative,Image_Only_Informative,Directed_Hate,Generalized_Hate,Sarcasm,Allegation,Justification,Refutation,Support,Oppose,favorite_count,possibly__senstive,retweet_count,user_verified,Processed_tweets
1191,"\there’s no obvious reason that a movement against misogyny, sexual assault, and non-consensual advances should be incompatible with fantasies of consensual, sexual submission.\"" why we still need erotica after #metoo: https://t.co/rsvkhjjzid https://t.co/zxdn8rflef""",1050642376836825090,1,0,1,0,0,1,0,0,1,0,4,0,1,1,no obvious reason that a movement against misogyny sexual assault and nonconsensual advances should be incompatible with fantasies of consensual sexual why we still need erotica after metoo URL URL
2261,after #metoo effect...😂😂😂 https://t.co/rsokxyqvxu,1052066153554538496,0,0,0,0,0,0,0,0,0,0,13,0,4,0,after metoo effect URL


In [ ]:
K = []
L = list(test_df['tweets'])
for i in range(len(L)):
  K.append(preprocess_tweet(L[i]))

test_df['Processed_tweets'] = pd.DataFrame(K)
test_df.sample(2)

,tweet_id,tweets,favorite_count,possibly__senstive,retweet_count,user_verified,Processed_tweets
1352,1056855755264065541,"@doctor_oxford in a fresh apology from a spa in arizona, philip green said that of course he'd meant his high court gag on the press to include a let-out for innocent banter. 'you just can't get the staff these days,' he lamented. 'or at least i can't.' #metoo https://t.co/anwoplsstx",0,0,0,0,USER_MENTION in a fresh apology from a spa in arizona philip green said that of course hed meant his high court gag on the press to include a letout for innocent banter you just cant get the staff these days he lamented or at least i cant metoo URL
438,1051850183506583552,the concerted efforts of the judiciary and movements like #metoo have definitely initiated a new age revolution where a woman does not just seek her rights but fights against every such act that encroaches upon her dignity. @nanagpatekar #tanushreedutta https://t.co/czqpfxyzkl https://t.co/ksq5yd9jfc,1,0,0,0,the concerted efforts of the judiciary and movements like metoo have definitely initiated a new age revolution where a woman does not just seek her rights but fights against every such act that encroaches upon her dignity USER_MENTION tanushreedutta URL URL


In [ ]:
E = []
L = list(train_df['emojis'])

for i in range(len(L)):
  E.append(emojis.count(L[i]))
train_df['emoji_count'] = pd.DataFrame(E)

In [ ]:
E = []
L = list(test_df['emojis'])

for i in range(len(L)):
  E.append(emojis.count(L[i]))
test_df['emoji_count'] = pd.DataFrame(E)

In [ ]:
def feat(tweet_df):
    ment=[]
    url=[]
    bg=[]
    for i in range(len(tweet_df)):
        ment.append(analyze_tweet(tweet_df["Processed_tweets"].values[i])[0]['MENTIONS'])
        url.append(analyze_tweet(tweet_df["Processed_tweets"].values[i])[0]['URLS'])
        bg.append(analyze_tweet(tweet_df["Processed_tweets"].values[i])[0]['BIGRAMS'])
    tweet_df['mentions']=ment
    tweet_df['url']=url
    tweet_df['bigrams']=bg
    return tweet_df

feat(train_df)
feat(test_df)

,tweet_id,tweets,favorite_count,possibly__senstive,retweet_count,user_verified,Processed_tweets,mentions,url,bigrams
0,1052098315687346177,such insensitive words. but alll laughing and applauding including the woman anchor. #metoo @dhanyarajendran https://t.co/aiplj7pnyj,552,0,188,0,such insensitive words but all laughing and applauding including the woman anchor metoo USER_MENTION URL,1,1,12
1,1052084623763591168,be like kali... supporting through art for #metoo #metooindia moment https://t.co/8o991kymub,0,0,1,0,be like kali supporting through art for metoo metooindia moment URL,0,1,9
2,1052089712004014083,men understand consent when a gay guy touches them: mallika dua on #metoo https://t.co/i9kxmkmd52 via @gayindia https://t.co/wuzk9uokxl,1,0,0,0,men understand consent when a gay guy touches them mallika dua on metoo URL via USER_MENTION URL,1,2,13
3,1052089572870385665,priya ramani is going to be fighting mj akbar’s 97 lawyers in the #metoo defamation suit. https://t.co/ipyelqz8wf,0,0,0,0,priya ramani is going to be fighting mj lawyers in the metoo defamation suit URL,0,1,13
4,1052089363667017728,must see this video. tells what is legal for #metoo @drabchandorkar @somanathg @proudbhagavathi @goldenthrust https://t.co/pkjr1auevn,1,0,0,0,must see this video tells what is legal for metoo USER_MENTION USER_MENTION USER_MENTION USER_MENTION URL,4,1,9
...,...,...,...,...,...,...,...,...,...,...
1732,1051214758370631680,saying #metoo changed their lives https://t.co/smllz1m3ez https://t.co/0bvwgirdmx,0,0,0,0,saying metoo changed their lives URL URL,0,2,4
1733,1050605251605094402,anil kapoor on #metoo: it is fantastic https://t.co/uqhbqfmwa5 https://t.co/z84zrha2t0,0,0,0,0,anil kapoor on metoo it is fantastic URL URL,0,2,6
1734,1049873692149391360,#metoo that ratham and thakkali chutney moment! https://t.co/cxpnvordpc,0,0,0,0,metoo that ratham and thakkali chutney moment URL,0,1,6
1735,1049904241165443077,coverage of #metoo and #vaw is excellent in ⁦@gulf_news⁩ - powerful stories of change all week. https://t.co/rfsapq4nqq,5,0,2,0,coverage of metoo and vaw is excellent in powerful stories of change all week URL,0,1,13


In [ ]:
import nltk
import string
import re

def clean_text(text):
    ## Remove puncuation
    text = text.translate(string.punctuation)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=_]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text

train_df['Processed_tweets'] = train_df['Processed_tweets'].map(lambda x: clean_text(x))
test_df['Processed_tweets'] = test_df['Processed_tweets'].map(lambda x: clean_text(x))

In [ ]:
train_df.sample(5)

,Tweets,TweetId,Text_Only_Informative,Image_Only_Informative,Directed_Hate,Generalized_Hate,Sarcasm,Allegation,Justification,Refutation,Support,Oppose,favorite_count,possibly__senstive,retweet_count,user_verified,Processed_tweets,mentions,url,bigrams
5809,this #metoo brigade causes shit like this. that poor kid. https://t.co/0kkg8iunts,1051020625307930624,1,0,0,0,0,0,0,0,1,0,1,0,3,0,this metoo brigade causes shit like this that poor kid URL,0,1,9
292,"it is not possible to be in favor of justice for some people and not be in favor for justice for all people- dr. martin luther king jr. just think about that, how these words still resonates today. #mentoo #kidstoo #metoo #survivorculture #abuseisgenderless #abusersaregenderless https://t.co/ckqhbpnzlh",1051961261607579648,0,0,0,0,0,0,0,0,0,0,4,0,6,0,it is not possible to be in favor of justice for some people and not be in favor for justice for all people dr martin luther king jr just think about that how these words still resonates today mentoo kidstoo metoo survivorculture abuseisgenderless abusersaregenderless URL,0,1,43
3288,after so long time i have wrote something.. time has changed.. now people must.. #metoo #metooindia #womenpower #feminist #feminism #womenempowerment #girls #woman #equality #respect… https://t.co/kzrcbaagxq https://t.co/5liedvi79f,1055790274461425664,1,0,0,0,0,0,0,0,0,1,0,0,0,0,after so long time i have wrote something time has changed now people must metoo metooindia womenpower feminist feminism womenempowerment girls woman equality URL URL,0,2,22
2609,currently india with #metooindia #metoo and #section497 #pun https://t.co/kr5qcgnvtq,1049672576195280897,1,0,0,1,0,0,0,0,0,1,0,0,0,0,currently india with metooindia metoo and section497 pun URL,0,1,7
6714,"watch @nehadhupia’s gentle, yet powerful take on the #metoo movement. #metooindia #metoomovement https://t.co/j07jojp8z7",1052132728550899712,0,0,0,0,0,0,0,0,0,0,10,0,1,1,watch USER_MENTION gentle yet powerful take on the metoo movement metooindia metoomovement URL,1,1,10


In [ ]:
train_df.columns

Index(['Tweets', 'TweetId', 'Text_Only_Informative', 'Image_Only_Informative',
       'Directed_Hate', 'Generalized_Hate', 'Sarcasm', 'Allegation',
       'Justification', 'Refutation', 'Support', 'Oppose', 'favorite_count',
       'possibly__senstive', 'retweet_count', 'user_verified',
       'Processed_tweets', 'mentions', 'url', 'bigrams'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df[['TweetId','Processed_tweets','favorite_count', 
                                                              'possibly__senstive', 'retweet_count', 'user_verified', 'mentions', 'url', 'bigrams']], 
                                                    train_df[['Text_Only_Informative', 'Image_Only_Informative',
       'Directed_Hate', 'Generalized_Hate', 'Sarcasm', 'Allegation',
       'Justification', 'Refutation', 'Support', 'Oppose']], test_size=0.2, random_state=42)

In [ ]:
X_train.sample(3)

,TweetId,Processed_tweets,favorite_count,possibly__senstive,retweet_count,user_verified,mentions,url,bigrams
619,1051479122986307584,pay it forward please help someone become a dental assistant educateallgirls educationforall educacion educationisgreat learning learningtogether metoo dental dentistry URL,0,0,0,0,0,1,18
5604,1051738896999088128,a nice lil ignorant comment to get me riled up on a monday morning metoo USER_MENTION URL,0,0,0,0,1,1,14
808,1051150624157560832,how has the metoo movement changed the world and what does that mean for writers join writers in residence at the university of iowa international writing program on october at noon in meeting room a to explore this topic USER_MENTION metoomovement genderroles URL,2,0,3,0,1,1,40


In [ ]:
#Copy the preprocessed tweets dataframe to .csv files
# train_df.to_csv('trainable.csv',index=False)
# test_df.to_csv('testable.csv',index=False)

In [ ]:
#CREATING A BUNCH OF ALL TWEETS (train, test) FOR CREATING EMBEDDINGS
all_tweets = pd.DataFrame(pd.concat([train_df['Processed_tweets'],test_df['Processed_tweets']],axis=0))
all_tweets.columns = ['tweets']
all_tweets['tweets'] = all_tweets['tweets'].astype(str)
all_tweets.sample(2)

,tweets
2061,metoo hashtag usage on USER_MENTION over the past year URL
2309,i had written metoo is created to downplay rapesonnuns by christian fathers bishops now it is clear that tanushreedutta is a convertedchristian rest is left to the readers URL


In [ ]:
list(all_tweets['tweets'])[0]

'what a shock hey old bill is in the clear every single imwithher metoo moron will swallow this huge pile of shit without a single gag hypocrites URL'

In [ ]:
dialogs=[]

for i in range(len(all_tweets)):
    my_string=list(all_tweets['tweets'])[i]
    tokens = word_tokenize(my_string)
    tokens= [word for word in tokens if word.isalnum()]
    words = [word.lower() for word in tokens]
    dialogs.append(words)

In [ ]:
corp=[dialogs[i] for i in range(len(dialogs))]

In [ ]:
#Finding the number of unique words in the whole corpus
flat_list = [item for sublist in corp for item in sublist]
print("Total number of words:", (len(flat_list)))
print("Total number of unique words: ",len(set(flat_list)))
#If stopwords are not removed, it's around 17,487

Total number of words: 183248
Total number of unique words:  17487


In [ ]:
# tweets with maximum lengths
print("training data tweet with maximum length : ",max(set(list(train_df['Processed_tweets'].str.split().str.len()))))
print("testing data tweet with maximum length : ",max(set(list(test_df['Processed_tweets'].str.split().str.len()))))

training data tweet with maximum length :  91
testing data tweet with maximum length :  90


### Word Tokenizer (Glove & Embeddings)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

--2020-07-29 17:12:50--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2020-07-29 17:12:51--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2020-07-29 17:12:51--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [ ]:
!unzip '/content/glove.twitter.27B.zip' -d '/content/glove.twitter.27B'

Archive:  /content/glove.twitter.27B.zip
  inflating: /content/glove.twitter.27B/glove.twitter.27B.25d.txt  
  inflating: /content/glove.twitter.27B/glove.twitter.27B.50d.txt  
  inflating: /content/glove.twitter.27B/glove.twitter.27B.100d.txt  
  inflating: /content/glove.twitter.27B/glove.twitter.27B.200d.txt  


##Bi-LSTM [for Stance, Hate labels]

In [ ]:
train = pd.concat([X_train[['Processed_tweets']], y_train],axis=1)
test = pd.concat([X_test[['Processed_tweets']], y_test],axis=1)
train.head(2)

,Processed_tweets,Text_Only_Informative,Image_Only_Informative,Directed_Hate,Generalized_Hate,Sarcasm,Allegation,Justification,Refutation,Support,Oppose
252,unsung hero rose schneiderman was a forerunner of the metoo movement history mondaymotivation URL URL,1,0,0,0,0,0,0,0,0,0
2705,metoo row nivetha pethuraj faces harrasement in party URL thimirupudichavan thimirupudichavanfromdiwali nivethapethuraj URL,1,1,0,0,0,0,0,0,0,0


In [ ]:
import re
def clean_text(text):
    text = re.sub(r'#(\S+)', r' \1 ', str(text))
    ## Remove punctuation
    text = text.translate(string.punctuation)
    text = re.sub(r"URL", "", str(text))
    text = re.sub(r"USER_MENTION", "", str(text))
    text = re.sub(r"\s{2,}", "", str(text))
    return text

train['Processed_tweets'] = train['Processed_tweets'].map(lambda x: clean_text(x))
test['Processed_tweets'] = test['Processed_tweets'].map(lambda x: clean_text(x))

categories = ['Directed_Hate', 'Generalized_Hate','Support', 'Oppose']

In [ ]:
train.head(2)

,Processed_tweets,Text_Only_Informative,Image_Only_Informative,Directed_Hate,Generalized_Hate,Sarcasm,Allegation,Justification,Refutation,Support,Oppose
252,unsung hero rose schneiderman was a forerunner of the metoo movement history mondaymotivation,1,0,0,0,0,0,0,0,0,0
2705,metoo row nivetha pethuraj faces harrasement in partythimirupudichavan thimirupudichavanfromdiwali nivethapethuraj,1,1,0,0,0,0,0,0,0,0


In [ ]:
# tweets with maximum lengths
print("training data tweet with maximum length : ",max(set(list(train['Processed_tweets'].str.split().str.len()))))
print("validation data tweet with maximum length : ",max(set(list(test['Processed_tweets'].str.split().str.len()))))

training data tweet with maximum length :  55
validation data tweet with maximum length :  53


In [ ]:
EMBEDDING_FILE = '/content/glove.twitter.27B/glove.twitter.27B.100d.txt'
def get_coefs(word,*arr):
  return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

In [ ]:
#CREATING A BUNCH OF ALL TWEETS (train, test) FOR CREATING EMBEDDINGS
all_tweets = pd.DataFrame(pd.concat([train['Processed_tweets'],test['Processed_tweets']],axis=0))
all_tweets.columns = ['tweets']
all_tweets['tweets'] = all_tweets['tweets'].astype(str)
all_tweets.sample(2)

,tweets
4996,metoo high court slams anurag kashyap and vikramaditya motwane for not producing witness
3257,kabir khan on metoo we ignored but it happen again


In [ ]:
from collections import Counter
results = Counter()
# total['tweet'].str.lower().str.split().apply(results.update)
all_tweets['tweets'].str.lower().str.split().apply(results.update)
print(results)
print("Size of train vocabulary: ",len(results.keys()))

Counter({'metoo': 6694, 'the': 4339, 'to': 3284, 'of': 2706, 'a': 2220, 'and': 2208, 'in': 1972, 'is': 1513, 'on': 1377, 'for': 1249, 'movement': 1011, 'this': 982, 'women': 925, 'sexual': 804, 'you': 771, 'with': 747, 'i': 708, 'are': 700, 'that': 662, 'be': 651, 'at': 621, 'it': 607, 'about': 577, 'not': 569, 'metooindia': 560, 'by': 546, 'against': 537, 'her': 521, 'from': 518, 'have': 505, 'we': 499, 'harassment': 493, 'as': 490, 'after': 456, 'has': 454, 'who': 449, 'all': 444, 'my': 371, 'but': 363, 'will': 362, 'what': 358, 'me': 355, 'more': 352, 'allegations': 344, 'she': 344, 'up': 338, 'out': 336, 'one': 335, 'your': 334, 'how': 324, 'was': 318, 'men': 313, 'our': 311, 'india': 294, 'their': 283, 'now': 276, 'no': 266, 'an': 266, 'they': 257, 'can': 257, 'says': 257, 'when': 257, 'he': 247, 'why': 246, 'over': 246, 'its': 242, 'so': 240, 'if': 239, 'his': 237, 'timesup': 233, 'woman': 227, 'or': 225, 'do': 223, 'year': 217, 'us': 215, 'been': 211, 'like': 208, 'accused': 207

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_tweets['tweets'])
max_length = 55
train_sequences = tokenizer.texts_to_sequences(train['Processed_tweets'])
train_data = pad_sequences(train_sequences, maxlen=max_length)

In [ ]:
vocabulary_size = 16500
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test['Processed_tweets'])
test_data = pad_sequences(test_sequences, maxlen=max_length)

In [ ]:
train_data.shape

(5495, 55)

In [ ]:
#Function for everything
from sklearn.metrics import roc_auc_score, f1_score
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from matplotlib import pyplot
#categories
preds = pd.DataFrame()
for i in categories:
  import random
  random.seed(35)
  #Callbacks
  lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=2, mode='max')
  model_glove = Sequential()
  model_glove.add(Embedding(vocabulary_size, 100, input_length=max_length, weights=[embedding_matrix], trainable=False))
  model_glove.add(Dropout(0.2))
  model_glove.add(Conv1D(64, 5, activation='relu'))
  model_glove.add(MaxPooling1D(pool_size=4))
  model_glove.add(Bidirectional(LSTM(100)))
  model_glove.add(Dense(1, activation='sigmoid'))
  model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics= [tf.keras.metrics.AUC()]) 
  print('BiLSTM fit for the label:',i)
  model_glove.fit(train_data, y_train[i].values,batch_size=32, epochs = 10,validation_split=0.1, callbacks=[lr_reduce])
  preds[i]= model_glove.predict_classes(test_data).ravel()
  print('Validation AUC for',i ,':',roc_auc_score(y_test[i].values,preds[i]))
  print('Validation F1-score for',i,':',f1_score(y_test[i].values,preds[i]))

BiLSTM fit for the label: Directed_Hate
Epoch 1/10
155/155 [==============================] - 2s 13ms/step - loss: 0.1769 - auc_29: 0.4798 - val_loss: 0.1448 - val_auc_29: 0.4886
Epoch 2/10
155/155 [==============================] - 1s 8ms/step - loss: 0.1594 - auc_29: 0.5483 - val_loss: 0.1471 - val_auc_29: 0.4116
Epoch 3/10
155/155 [==============================] - 1s 8ms/step - loss: 0.1528 - auc_29: 0.6700 - val_loss: 0.1601 - val_auc_29: 0.3673
Epoch 4/10
155/155 [==============================] - 1s 8ms/step - loss: 0.1363 - auc_29: 0.8029 - val_loss: 0.1735 - val_auc_29: 0.3945
Epoch 5/10
155/155 [==============================] - 1s 9ms/step - loss: 0.1087 - auc_29: 0.9051 - val_loss: 0.1923 - val_auc_29: 0.3789
Epoch 6/10
155/155 [==============================] - 1s 9ms/step - loss: 0.0775 - auc_29: 0.9594 - val_loss: 0.2482 - val_auc_29: 0.3513
Epoch 7/10
155/155 [==============================] - 1s 8ms/step - loss: 0.0679 - auc_29: 0.9707 - val_loss: 0.2576 - val_auc_29: 

## LGBM

In [ ]:
import lightgbm as lgbm

categories = ['Text_Only_Informative','Image_Only_Informative','Directed_Hate', 'Generalized_Hate', 'Sarcasm', 'Allegation','Justification', 'Refutation','Support', 'Oppose']

X_train = train_data
X_test = test_data
pred = pd.DataFrame()

for category in categories:
  print('\n\n')
  print('LightGBM Fit for the label: ',category)

  #scale_pos_weights = sum(negative instances) / sum(positive instances)
  cls_weights = (len(y_train)-y_train[category].sum())/(y_train[category].sum())
  m = lgbm.LGBMClassifier(objective='binary',max_depth=3,learning_rate=0.01,scale_pos_weight=round(cls_weights,2),boosting_type='gbdt',metric='binary_logloss', 
                     random_state=5,reg_lambda=20,silent=False)
  
  m.fit(X_train,y_train[category].values)
  pred[category] = m.predict(X_test)
  print('Validation AUC for',category,':',roc_auc_score(y_test[category].values,pred[category]))
  print('Validation F1-score for',category,':',f1_score(y_test[category].values,pred[category]))

In [ ]:
from sklearn.metrics import roc_auc_score
for i in ['Directed_Hate', 'Generalized_Hate', 'Sarcasm', 'Allegation','Justification', 'Refutation','Support', 'Oppose']:
  print('AUC for label ',i ,':',roc_auc_score(y_test[i],pred[i]))

AUC for label  Directed_Hate : 0.491016044858264
AUC for label  Generalized_Hate : 0.5134008243392539
AUC for label  Sarcasm : 0.4848860632003022
AUC for label  Allegation : 0.4913763806287171
AUC for label  Justification : 0.4944509345794392
AUC for label  Refutation : 0.5097333955684906
AUC for label  Support : 0.5042424965592545
AUC for label  Oppose : 0.4979161124412521


##XGBOOST

In [ ]:
final_train_xgb = pd.concat([pd.DataFrame(train_data),X_train[['favorite_count','possibly__senstive','retweet_count','user_verified','mentions','url','bigrams']]],axis=1)
final_test_xgb = pd.concat([pd.DataFrame(test_data),X_test[['favorite_count','possibly__senstive','retweet_count','user_verified','mentions','url','bigrams']]],axis=1)

In [ ]:
import xgboost as xgb
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

categories = ['Text_Only_Informative', 'Image_Only_Informative','Directed_Hate', 'Generalized_Hate', 'Sarcasm', 'Allegation','Justification', 'Refutation', 'Support', 'Oppose']

pred = pd.DataFrame()

for category in categories:
  #scale_pos_weights = sum(negative instances) / sum(positive instances)
  cls_weights = (len(y_train)-y_train[category].sum())/(y_train[category].sum())
  clf = xgb.XGBClassifier(scale_pos_weight =round(cls_weights,2))
  clf.fit(train_data,y_train[category].values)
  print('... Processing XGBoost Classifier for {}'.format(category))
  pred[category] = clf.predict(test_data)
  print("Validation AUC-ROC Score: ", roc_auc_score(y_test[category],pred[category]))
  print("Validation F1 Score: ", f1_score(y_test[category],pred[category]))

# #Confusion Matrix
# print('\n')
# print('CONFUSION MATRIX:->')
# skplt.metrics.plot_confusion_matrix(val_Y,np.round(xgb_pred),figsize=(6,6))

... Processing XGBoost Classifier for Text_Only_Informative
Validation AUC-ROC Score:  0.5099565931775265
Validation F1 Score:  0.6854709952102181
... Processing XGBoost Classifier for Image_Only_Informative
Validation AUC-ROC Score:  0.539382634915726
Validation F1 Score:  0.3940990516332982
... Processing XGBoost Classifier for Directed_Hate
Validation AUC-ROC Score:  0.5026515151515152
Validation F1 Score:  0.06521739130434784
... Processing XGBoost Classifier for Generalized_Hate
Validation AUC-ROC Score:  0.5239598811292719
Validation F1 Score:  0.04560260586319218
... Processing XGBoost Classifier for Sarcasm
Validation AUC-ROC Score:  0.5350658956428187
Validation F1 Score:  0.04519774011299435
... Processing XGBoost Classifier for Allegation
Validation AUC-ROC Score:  0.47183075129102114
Validation F1 Score:  0.04580152671755725
... Processing XGBoost Classifier for Justification
Validation AUC-ROC Score:  0.5053038239916187
Validation F1 Score:  0.05809128630705394
... Process

##RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

categories = ['Text_Only_Informative', 'Image_Only_Informative','Directed_Hate', 'Generalized_Hate', 'Sarcasm', 'Allegation','Justification', 'Refutation', 'Support', 'Oppose']

pred = pd.DataFrame()

for category in categories:
  #Balanced Class weight
  #classes are automatically weighted inversely proportional to how frequently they appear in the data
  clf = RandomForestClassifier(class_weight="balanced")
  clf.fit(train_data,y_train[category].values)
  print('... Processing RF Classifier for {}'.format(category))
  pred[category] = clf.predict(test_data)
  print("Validation AUC-ROC Score: ", roc_auc_score(y_test[category],pred[category]))
  print("Validation F1 Score: ", f1_score(y_test[category],pred[category]))

... Processing RF Classifier for Text_Only_Informative
Validation AUC-ROC Score:  0.5046186167038609
Validation F1 Score:  0.8203776899429073
... Processing RF Classifier for Image_Only_Informative
Validation AUC-ROC Score:  0.5011138278954693
Validation F1 Score:  0.1354723707664884
... Processing RF Classifier for Directed_Hate
Validation AUC-ROC Score:  0.5128367003367004
Validation F1 Score:  0.05633802816901408
... Processing RF Classifier for Generalized_Hate
Validation AUC-ROC Score:  0.5279133941838251
Validation F1 Score:  0.0784313725490196
... Processing RF Classifier for Sarcasm
Validation AUC-ROC Score:  0.4988905325443787
Validation F1 Score:  0.0
... Processing RF Classifier for Allegation
Validation AUC-ROC Score:  0.5144594369481926
Validation F1 Score:  0.06593406593406594
... Processing RF Classifier for Justification
Validation AUC-ROC Score:  0.5074810110005239
Validation F1 Score:  0.03571428571428571
... Processing RF Classifier for Refutation
Validation AUC-ROC 

##MNB

In [ ]:
X_train_NB = X_train.Processed_tweets
X_val_NB = X_test.Processed_tweets
print(X_train_NB.shape)
print(X_val_NB.shape)

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train_NB)
print('Shape using CountVectorizer():' , X_train_counts.shape)


from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('Shape using TfidfTransformer():' , X_train_tfidf.shape)

(5495,)
(1374,)
Shape using CountVectorizer(): (5495, 13194)
Shape using TfidfTransformer(): (5495, 13194)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])

prediction = pd.DataFrame()
for category in categories:
    print('... Processing {}'.format(category))
    text_clf.fit(X_train_NB, y_train[category])
    # compute the validation accuracy
    prediction[category] = text_clf.predict(X_val_NB)
    print("Validation AUC-ROC Score: ", roc_auc_score(y_test[category],prediction[category]))
    print('Validation F1-Score using naive_bayes: ', f1_score(y_test[category],prediction[category]))
    #Confusion Matrix
    # print('\n')
    # print('CONFUSION MATRIX:->')
    # skplt.metrics.plot_confusion_matrix(val_Y[category],np.round(prediction),figsize=(6,6))

... Processing Text_Only_Informative
Validation AUC-ROC Score:  0.5058979326581161
Validation F1-Score using naive_bayes:  0.8486122792262405
... Processing Image_Only_Informative
Validation AUC-ROC Score:  0.4990963738982527
Validation F1-Score using naive_bayes:  0.017391304347826084
... Processing Directed_Hate
Validation AUC-ROC Score:  0.5
Validation F1-Score using naive_bayes:  0.0
... Processing Generalized_Hate
Validation AUC-ROC Score:  0.5
Validation F1-Score using naive_bayes:  0.0
... Processing Sarcasm
Validation AUC-ROC Score:  0.5
Validation F1-Score using naive_bayes:  0.0
... Processing Allegation
Validation AUC-ROC Score:  0.5
Validation F1-Score using naive_bayes:  0.0
... Processing Justification
Validation AUC-ROC Score:  0.5
Validation F1-Score using naive_bayes:  0.0
... Processing Refutation
Validation AUC-ROC Score:  0.5
Validation F1-Score using naive_bayes:  0.0
... Processing Support
Validation AUC-ROC Score:  0.4956803455723542
Validation F1-Score using nai

##SVM

In [ ]:
# X_train = pd.read_csv('/content/train_set.csv') 
# y_train = pd.read_csv('/content/train_labels.csv')
# X_test = pd.read_csv('/content/test_set.csv')
# y_test = pd.read_csv('/content/test_labels.csv')

In [ ]:
X_train.head(2)

,TweetId,Processed_tweets,favorite_count,possibly__senstive,retweet_count,user_verified,mentions,url,bigrams
0,1051957160182259712,unsung hero rose schneiderman was a forerunner of the metoo movement history mondaymotivation URL URL,0,0,0,0,0,2,12
1,1056816001851904001,metoo row nivetha pethuraj faces harrasement in party URL thimirupudichavan thimirupudichavanfromdiwali nivethapethuraj URL,7,0,0,1,0,2,10


In [ ]:
import re
def clean_text(text):
    text = re.sub(r'#(\S+)', r' \1 ', str(text))
    ## Remove punctuation
    text = text.translate(string.punctuation)
    text = re.sub(r"URL", "", str(text))
    text = re.sub(r"USER_MENTION", "", str(text))
    text = re.sub(r"\s{2,}", "", str(text))
    return text

X_train['Processed_tweets'] = X_train['Processed_tweets'].map(lambda x: clean_text(x))
X_test['Processed_tweets'] = X_test['Processed_tweets'].map(lambda x: clean_text(x))

In [ ]:
#pipeline of feature engineering and model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),('standardscaler', StandardScaler(with_mean=False)),
    ('clf', SVC(class_weight="balanced"))])

In [ ]:
model

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabula...
                                  sublinear_tf=False, use_idf=True)),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight='balanced', coef0=0.0,
            

In [ ]:
categories = ['Text_Only_Informative', 'Image_Only_Informative', 'Directed_Hate',
       'Generalized_Hate', 'Sarcasm', 'Allegation', 'Justification',
       'Refutation', 'Support', 'Oppose']
prediction = pd.DataFrame()
for category in categories:
    print('... Processing {}'.format(category))
    model.fit(X_train['Processed_tweets'], y_train[category])
    # compute the validation metrics
    prediction[category] = model.predict(X_test['Processed_tweets'])
    print("Validation AUC-ROC Score: ", roc_auc_score(y_test[category],prediction[category]))
    print('Validation F1-Score using naive_bayes: ', f1_score(y_test[category],prediction[category]))

... Processing Text_Only_Informative
Validation AUC-ROC Score:  0.5178205033883481
Validation F1-Score using naive_bayes:  0.7963218390804598
... Processing Image_Only_Informative
Validation AUC-ROC Score:  0.5095267318694913
Validation F1-Score using naive_bayes:  0.19110378912685336
... Processing Directed_Hate
Validation AUC-ROC Score:  0.5109427609427609
Validation F1-Score using naive_bayes:  0.05263157894736842
... Processing Generalized_Hate
Validation AUC-ROC Score:  0.5245701549564847
Validation F1-Score using naive_bayes:  0.06666666666666667
... Processing Sarcasm
Validation AUC-ROC Score:  0.496301775147929
Validation F1-Score using naive_bayes:  0.0
... Processing Allegation
Validation AUC-ROC Score:  0.5159586873230052
Validation F1-Score using naive_bayes:  0.07476635514018691
... Processing Justification
Validation AUC-ROC Score:  0.5044689628077528
Validation F1-Score using naive_bayes:  0.03125
... Processing Refutation
Validation AUC-ROC Score:  0.5465196769644918
Va

##FASTTEXT

In [ ]:
train = pd.read_csv('trainable.csv')
test = pd.read_csv('testable.csv')
train.head(2)

,Tweets,Text_Only_Informative,Image_Only_Informative,Directed_Hate,Generalized_Hate,Sarcasm,Allegation,Justification,Refutation,Support,Oppose,favorite_count,possibly__senstive,retweet_count,user_verified,emojis,Processed_tweets,emoji_count,mentions,url,bigrams,cleaned_tweets
0,"what a shock hey, old bill is in the clear &amp; every single #imwithher #metoo moron will swallow this huge pile of shit without a single gag 🤷🏼‍♂️ #hypocrites https://t.co/xkymfqnanj",1,1,1,0,0,1,1,0,0,0,1,0,0,0,🤷🏼♂,what a shock hey old bill is in the clear every single imwithher metoo moron will swallow this huge pile of shit without a single gag hypocrites URL,1,0,1,26,shock hey old bill clear every single imwithher metoo moron swallow huge pile shit without single gag hypocrites url
1,"no bucky, people are pissed off at ashleyperkins89 for attacking the credibility of an assault survivor after the attacker, thomas ravenel, was arrested as the result of a four month police investigation. the only people questioning the #metoo movement are men and rape enablers. https://t.co/yaqrgivqb3",1,1,0,0,0,0,1,0,1,0,6,0,1,0,NaN,no bucky people are pissed off at ashleyperkins89 for attacking the credibility of an assault survivor after the attacker thomas ravenel was arrested as the result of a four month police investigation the only people questioning the metoo movement are men and rape enablers URL,0,0,1,43,bucky people pissed ashleyperkins89 attacking credibility assault survivor attacker thomas ravenel arrested result four month police investigation people questioning metoo movement men rape enablers url


In [ ]:
import seaborn as sns
categories = ['Text_Only_Informative', 'Image_Only_Informative','Directed_Hate', 'Generalized_Hate', 'Sarcasm', 'Allegation','Justification', 'Refutation', 'Support', 'Oppose']
counts = []
for i in categories:
    counts.append((i, train[i].sum()))
df_stats = pd.DataFrame(counts[:10], columns=['category', 'number_of_comments'])
df_stats['class_ratio'] = round(df_stats['number_of_comments']/float(len(train)),2)

print(df_stats)

import matplotlib.pyplot as plt
%matplotlib inline 
fig = plt.figure(figsize = (15, 10)) 
# creating the bar plot 
plt.bar(df_stats.category.values, df_stats.class_ratio.values, color ='maroon',  
        width = 0.4)

In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/v0.1.0.zip

--2020-07-28 18:30:09--  https://github.com/facebookresearch/fastText/archive/v0.1.0.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.1.0 [following]
--2020-07-28 18:30:10--  https://codeload.github.com/facebookresearch/fastText/zip/v0.1.0
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.1.0.zip’

v0.1.0.zip              [  <=>               ]  92.06K   173KB/s    in 0.5s    

2020-07-28 18:30:11 (173 KB/s) - ‘v0.1.0.zip’ saved [94267]



In [ ]:
!unzip v0.1.0.zip

In [ ]:
cd /content/fastText-0.1.0

/content/fastText-0.1.0


In [ ]:
!make

c++ -pthread -std=c++0x -O3 -funroll-loops -c src/args.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/dictionary.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/productquantizer.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/matrix.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/qmatrix.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/vector.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/model.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/utils.cc
c++ -pthread -std=c++0x -O3 -funroll-loops -c src/fasttext.cc
c++ -pthread -std=c++0x -O3 -funroll-loops args.o dictionary.o productquantizer.o matrix.o qmatrix.o vector.o model.o utils.o fasttext.o src/main.cc -o fasttext


In [ ]:
!./fasttext

usage: fasttext <command> <args>

The commands supported by fasttext are:

  supervised              train a supervised classifier
  quantize                quantize a model to reduce the memory usage
  test                    evaluate a supervised classifier
  predict                 predict most likely labels
  predict-prob            predict most likely labels with probabilities
  skipgram                train a skipgram model
  cbow                    train a cbow model
  print-word-vectors      print word vectors given a trained model
  print-sentence-vectors  print sentence vectors given a trained model
  nn                      query for nearest neighbors
  analogies               query for analogies



In [ ]:
train = train[['Support','Processed_tweets']]
import pandas as pd
from io import StringIO
import csv
train['Support'] = train['Support'].map({1: 'positive', 0: 'negative'})

In [ ]:
test['Processed_tweets']= test['Processed_tweets'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
test.to_csv(r'test1.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [ ]:
test.head(1)

,Support,Processed_tweets
0,__label__negative,what a shock hey old bill is in the clear every single imwithher metoo moron will swallow this huge pile of shit without a single gag hypocrites URL


In [ ]:
col = ['Support','Processed_tweets']
train['Support']=['__label__'+ s for s in train['Support']]
train['Processed_tweets']= train['Processed_tweets'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
train.to_csv(r'support_updated.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [ ]:
!head -n 6000 "/content/fastText-0.1.0/support_updated.txt" > "/content/fastText-0.1.0/support_updated_train.txt"
!tail -n 869 "/content/fastText-0.1.0/support_updated.txt" > "/content/fastText-0.1.0/support_updated_test.txt"

In [ ]:
%%time
!./fasttext supervised -input "/content/fastText-0.1.0/support_updated.txt" -output model_support

Read 0M words
Number of words:  15027
Number of labels: 2
Progress: 100.0%  words/sec/thread: 3220753  lr: 0.000000  loss: 0.655859  eta: 0h0m 
CPU times: user 19.5 ms, sys: 10.4 ms, total: 29.9 ms
Wall time: 4.21 s


In [ ]:
!./fasttext test model_support.bin "/content/fastText-0.1.0/support_updated_test.txt"

N	869
P@1	0.702
R@1	0.702
Number of examples: 869


In [ ]:
A = !./fasttext predict model_support.bin "/content/fastText-0.1.0/test1.txt"

In [ ]:
x = pd.DataFrame(A)
x.columns = ['Predicted_S']

In [ ]:
x['Predicted_S'] = x['Predicted_S'].map({'__label__negative': 0, '__label__positive': 1})